In [ ]:
!pwd

In [79]:
import torch
import matplotlib.pylab as plt
import matplotlib.patches as patches
import numpy as np
from utils.mydataset import MyDataset

In [ ]:
data_dir = 'dataset'
dataset = MyDataset(data_dir=data_dir)

In [99]:
dataset = MyDataset(data_dir = 'adjusted_100_dataset')

In [96]:
dataset.all_data_dir_list

['adjusted_100_dataset/a073_kp/S028C002P087R001A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S027C001P080R001A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S031C003P067R002A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S028C001P073R002A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S026C002P008R001A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S030C003P075R002A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S019C002P041R002A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S029C003P067R002A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S026C001P044R002A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S026C003P050R002A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S019C001P047R001A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S023C003P060R002A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S019C003P046R001A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S018C002P041R002A073_rgb_kp.json',
 'adjusted_100_dataset/a073_kp/S021C001P055R001A073_rgb_kp.json',
 'adjusted

In [101]:
len(dataset.action_dir_dic.keys())

73

In [ ]:
dataset.all_data_dir_list[0], dataset.all_data_dir_list[90], dataset.all_data_dir_list[10201], dataset.all_data_dir_list[56704]

In [84]:
data_item = dataset[100]
data_item.keys()
# data_item['semi_positives_c0'][0].size()

dict_keys(['anchor', 'anchor_heatmap', 'anchor_flow', 'class'])

In [ ]:
for key in data_item.keys():
    if key == 'class':
        print(key)
        print(data_item[key])
    else:
        print(key)
        print(data_item[key].size())
        print(data_item[key].max())
        print(data_item[key].min())

In [ ]:
data_item['heatmap'].size()

In [ ]:
data_item['optical_flow'].size()

In [ ]:
heatmap = data_item['heatmap']
optical_flow = data_item['optical_flow']
origin = data_item['origin']

In [ ]:
origin.size()

In [ ]:
print(optical_flow[:,:,[0],:,:].sum(dim=1).reshape(-1,270,480)[0].max())
print(optical_flow[:,:,[0],:,:].sum(dim=1)[0].max())
print(optical_flow[:,:,[0],:,:].sum(dim=1)[1].max())

In [ ]:
print(heatmap[0].sum(dim=[0]).shape)
print(optical_flow[:,:,[0],:,:][0].sum(dim=[0,1]).shape)
print(heatmap.sum(dim=1).shape)
print(optical_flow[:,:,[0],:,:].sum(dim=[1,2]).shape)

In [ ]:
def flow_viz(heatmap, optical_flow):
    heatmap_vis = heatmap.sum(dim=1)
    # optical_flow_x = optical_flow[:,:,[0],:,:].sum(dim=[1,2])
    # optical_flow_y = optical_flow[:,:,[1],:,:].sum(dim=[1,2])
    optical_flow_x = optical_flow[:,:,[0],:,:]
    optical_flow_y = optical_flow[:,:,[1],:,:]

    i,j = np.meshgrid(np.arange(heatmap.sha))
    for i in range(1, len(heatmap_vis)):
        heat = heatmap_vis[i]
        u = optical_flow_x[i-1]
        v = optical_flow_y[i-1]
        plt.imshow(heat)
        plt.quiver(heat,u,v)
        plt.show()

In [ ]:
def flow_viz(heatmap, optical_flow):
    B,J,H,W = heatmap.shape

    optical_flow_x = optical_flow[:,:,[0],:,:]
    optical_flow_y = optical_flow[:,:,[1],:,:]

    x,y = np.meshgrid(np.arange(W, dtype=np.float32), np.arange(H, dtype=np.float32), indexing='xy')
    print(optical_flow_x.shape, x.shape)
    for i in range(1, B):
        plt.subplot(121)
        plt.imshow(heatmap[i-1][5])
        
        plt.subplot(122)
        plt.imshow(heatmap[i][5])
        # u = optical_flow_x[i-1][7].squeeze()
        # v = optical_flow_y[i-1][7].squeeze()
        # plt.quiver(x,y,u,v)
        plt.show()

In [ ]:
origin.shape

In [ ]:
keypoints = origin[:, 5,:]

flow = keypoints[1:] - keypoints[:-1]
flow.shape

In [ ]:
flow[0][0]

In [ ]:
origin

In [ ]:
u

In [ ]:
v

In [ ]:
u.shape, v.shape

In [ ]:
v

In [ ]:
u

In [ ]:
origin

In [ ]:
# 5:R_shoulder
keypoints = origin[:, 5,:] # [12,17,2]
flow = keypoints[:-1] - keypoints[1:]

flowmap = torch.zeros((11, 2, 270, 480))
for i in range(len(flow)):
    flowmap[i][0] = flowmap[i][0] + flow[i][0]
    flowmap[i][1] = flowmap[i][1] + flow[i][1]

In [ ]:
flow

In [ ]:
keypoints

In [ ]:
from torch.nn import functional as F

i,j = np.meshgrid(np.arange(480, dtype=np.float32), np.arange(270, dtype=np.float32))

u = flowmap[0,0]
v = flowmap[0,1]

# flowmap_normalized = F.normalize(flowmap, dim=1)
# u = flowmap_normalized[0,0]
# v = flowmap_normalized[0,1]

u_vis = torch.zeros_like(u, requires_grad=False)
v_vis = torch.zeros_like(v, requires_grad=False)
# u_vis[::10, ::10] = u[::10, ::10]
# v_vis[::10, ::10] = v[::10, ::10]

plt.subplot(121)
plt.imshow(heatmap[0][5])
plt.subplot(122)
plt.imshow(heatmap[1][5])

uu = u * heatmap[1][5]
vv = v * heatmap[1][5]
r, w = 10, 10
# plt.quiver(i,j,uu,vv, scale=5)
plt.quiver(i[::r, ::w],j[::r, ::w],uu[::r, ::w],vv[::r, ::w])

plt.show()

# plt.quiver(i,j,u_vis,v_vis)
# plt.show()

In [ ]:
u * heatmap[1][5]

In [ ]:
flow

In [ ]:
flowmap 

In [ ]:
heatmap.shape

In [ ]:
optical_flow[0][0][0]

In [ ]:
flow_viz(heatmap=heatmap, optical_flow=optical_flow)

In [ ]:
def heatmap_viz(heatmap):

    heatmap_vis = heatmap.sum(dim=1)
    
    for i in range(len(heatmap)):
        # tmp_heatmap = torch.zeros(heatmap.shape[2:4])
        # for j in range(len(heatmap[i])):
        #     tmp_heatmap += heatmap[i][j]
            # if j == 0:
            #     tmp_heatmap = heatmap[i][j]
            # else:
            #     tmp_heatmap += heatmap[i][j]
        
        # plt.imshow(tmp_heatmap)
        plt.imshow(heatmap_vis[i])
        plt.show()

In [ ]:
heatmap_viz(data_item['heatmap'])

In [ ]:
heatmap_viz(data_item['heatmap'])

In [ ]:
heatmap = data_item['heatmap']

In [ ]:
flow_1_0 = (heatmap[1] - heatmap[0]) * heatmap[1]
x = flow_1_0[0].max(dim=1)[1].max()
y = flow_1_0[0].max(dim=0)[1].max()
flow_x = (flow_1_0[0] + x) * heatmap[1][0]
flow_y = (flow_1_0[0] + y) * heatmap[1][0]
print(flow_x.max())
print(flow_y.max())

In [ ]:
plt.imshow(flow_x)

In [ ]:
plt.imshow(flow_y)

In [ ]:
for i in range(5):
    print(f"anchor_c{i}", data_item[f"anchor_c{i}"].shape)
    print(f"anchor_aug_c{i}", data_item[f"anchor_aug_c{i}"].shape)
    print(f"semi_positives_c{i}", data_item[f"semi_positives_c{i}"].shape)

In [ ]:
import os

In [ ]:
def visualize_keypoint(c0, c1, c2, c3, c4, a):
    __, K, D = c0.shape
    os.mkdir('test/semi/'+a.split('/')[-1].split('.')[0])
    
    neck_keypoint = np.array([5.0, 5.0]).reshape(1, 1, 2)
    neck_keypoint = np.tile(neck_keypoint, [1, K, 1])
    
    nose = c0[:, :, :2] + neck_keypoint
    left_shoulder = c0[:, :, 2:4] + neck_keypoint
    right_shoulder = c0[:, :, 4:6] + neck_keypoint
    mid_hip = c0[:, :, 6:8] + neck_keypoint
    left_hip = c0[:, :, 8:10] + mid_hip
    right_hip = c0[:, :, 10:12] + mid_hip

    left_elbow = c1[:, :, 0:2]  + left_shoulder
    left_wrist = c1[:, :, 2:4] + left_elbow

    right_elbow = c2[:, :, 0:2]  + right_shoulder
    right_wrist = c2[:, :, 2:4] + right_elbow

    left_knee = c3[:, :, 0:2] + left_hip
    left_ankle = c3[:, :, 2:4] + left_knee

    right_knee = c4[:, :, 0:2] + right_hip
    right_ankle = c4[:, :, 2:4] + right_knee

    restored_keypoints = np.stack([
        nose,
        neck_keypoint,
        left_shoulder,
        right_ankle,
        mid_hip,
        left_hip,
        right_hip,
        left_elbow,
        left_wrist,
        right_elbow,
        right_wrist,
        left_knee,
        left_ankle,
        right_knee,
        right_ankle
    ], axis=-2)

    print(restored_keypoints.shape)
    print(restored_keypoints.max(), restored_keypoints.min())

    for frame_index in range(restored_keypoints.shape[1]):
        canvas = np.zeros((10, 10))

        fig, ax = plt.subplots(1, 1)
        ax.imshow(canvas)

        for keypoint_index in range(restored_keypoints.shape[2]):
            ax.add_patch(patches.Circle(
                [restored_keypoints[0, frame_index, keypoint_index][1], restored_keypoints[0, frame_index, keypoint_index][0]],
                radius=0.1,
                fill=True,
                color="red"
            ))
        
        fn = 'test/semi/'+a.split('/')[-1].split('.')[0]+'/'+str(frame_index)
        plt.savefig(fn)
        plt.show()

In [ ]:
def test_anchor(index):
    data_item = dataset[index]
    visualize_keypoint(
        *[data_item[f"anchor_c{i}"] for i in range(5)]
    )

In [ ]:
def test_anchor_aug(index):
    a, data_item = dataset[index]
    visualize_keypoint(
        *[data_item[f"anchor_aug_c{i}"] for i in range(5)], a
    )
    print(a)

In [ ]:
import tensorflow as tf

In [ ]:
def test_semi(index):
    a, data_item = dataset[index]
    for j in range(len(a)):
        visualize_keypoint(
            *[data_item[f"semi_positives_c{i}"][j][tf.newaxis,:,:] for i in range(5)], a[j]
        )
        print(a[j])

In [ ]:
a, data_item = dataset[0]
print(a)

In [ ]:
dataset.all_data_dir_list[0], dataset.all_data_dir_list[90], dataset.all_data_dir_list[2345], dataset.all_data_dir_list[10201], dataset.all_data_dir_list[56704]

In [ ]:
test_semi(3444)

In [ ]:
import os
import numpy as np
import json
import torch
import math

In [ ]:
def extract_keypoints_from_a_json(origin_vid_dir):
    with open(origin_vid_dir, "r") as f:
        origin_keypoints = json.load(f)

    origin_keypoints_anno = origin_keypoints['annotations']
    origin_tmp_list = []
    for frame_data in origin_keypoints_anno:
        if frame_data:
            keypoints_list = [frame_data[key] for key in frame_data if frame_data[key]]
            if keypoints_list:
                origin_tmp_list.extend(keypoints_list)

    # # Nonetype error 발생하는 듯
    # if not origin_tmp_list:
    #     return None

    # reshape x
    origin_anchor_keypoints = np.array(origin_tmp_list)[:,:,[0,1]]
    return origin_anchor_keypoints

In [ ]:
def gaussian(points, h, w , sigma):
    x = points[:,[1]][0]
    y = points[:,[0]][0]
    channel = [np.exp(-((w - x) ** 2 + (h - y) ** 2) / (2 * sigma ** 2))]
    # channel = np.reshape(channel, newshape=(h,w))
    return channel

In [ ]:
data = extract_keypoints_from_a_json('dataset/a001_kp/S001C001P001R001A001_rgb_kp.json')
points = data.reshape(-1,2)

h = 1080
w = 1920

sigma = 5

channel = gaussian(points, h, w, sigma)

print(channel)

# x_points = x.reshape(-1,)
# y_points = y.reshape(-1,)

# x_gau = gaussian(x_points, mean, sigma)
# mu_x = 0.5 * (x + 1.) * w
# mu_y = 0.5 * (y + 1.) * h

# tmp_size = sigma * 3

# x1, y1 = mu_x - tmp_size, mu_y - tmp_size

# print(x_gau)

In [ ]:
# N, H, W = 12, 10, 10
# landmarks = torch.randint(0, H, (N, 2))

# tmp = torch.zeros(N, 3, H, W)
# rows, cols = landmarks.split(1, 1)
# tmp[torch.arange(N), :, rows, cols] = 1.

# image = torch.randn(N, 3, H, W)
# image = torch.cat((image, tmp), 1)
# print(image.shape)
# > torch.Size([12, 6, 10, 10])

np.transpose()

In [ ]:
kp = (100, 150) # W,H
kp2 = (100, 180)
H = np.tile(np.arange(256), 256).reshape(256,256).T
W = np.tile(np.arange(256), 256).reshape(256,256)

std = 5
x_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((H-kp[1])/std)**2) 
y_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((W-kp[0])/std)**2)
frame1 = 1000*(x_gauss * y_gauss)

print(W)

In [ ]:
def extract_keypoints_from_a_json(origin_vid_dir):
    with open(origin_vid_dir, "r") as f:
        origin_keypoints = json.load(f)

    origin_keypoints_anno = origin_keypoints['annotations']
    origin_tmp_list = []
    for frame_data in origin_keypoints_anno:
        if frame_data:
            keypoints_list = [frame_data[key] for key in frame_data if frame_data[key]]
            if keypoints_list:
                origin_tmp_list.extend(keypoints_list)

    # # Nonetype error 발생하는 듯
    # if not origin_tmp_list:
    #     return None

    # reshape x
    origin_anchor_keypoints = np.array(origin_tmp_list)[:,:,[0,1]]
    return origin_anchor_keypoints

In [ ]:
import numpy as np
import json

data = extract_keypoints_from_a_json('dataset/a001_kp/S001C001P001R001A001_rgb_kp.json')
points = data.reshape(-1,2)
x_points = points[:,[0]]
y_points = points[:,[1]]

h = 1080
w = 1920
# w = 1920*(1080/1920)

H = np.tile(np.arange(h), h).reshape(h,h).T
print(H)
print(H.shape)
W = np.tile(np.arange(w), w).reshape(w,w)
print(W.shape)

std = 5
x = x_points[0]
y = y_points[0]*(1080/1920)
x_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((H-x)/std)**2) 
y_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((W-y)/std)**2)
frame1 = 1000*(x_gauss * y_gauss)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from PIL import Image

data = extract_keypoints_from_a_json('dataset/a001_kp/S001C001P001R001A001_rgb_kp.json')
points = data.reshape(-1,2)
x_points = points[:,[0]]
y_points = points[:,[1]]

h = 1080
w = 1920
# w = 1920*(1080/1920)

H = np.tile(np.arange(h)[:, None], (h, w))
print(H)
print(H.shape)
W = np.tile(np.arange(w)[None, :], (h, w))

std = 5

# org_image = Image.open('frame0.jpg')
# resized_image = org_image.resize((int(w), h))

for i in range(17):
    x = x_points[i]
    y = y_points[i]
    # y = y_points[i]*(1080/1920)
    x_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((H-x)/std)**2) 
    y_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((W-y)/std)**2)
    frame = (x_gauss * y_gauss)

    # plt.imshow(resized_image)
    # plt.imshow(frame, alpha = 0.6, cmap = 'viridis')
    plt.imshow(frame)
    plt.savefig(f'heatmap/A001{i}.png', bbox_inches='tight', pad_inches=0)
    # overlap = frame * 0.4 + resized_image

    # overlap_pil = Image.fromarray(overlap)
    # overlap_pil.show()

    # overlap_pil.save(f'overlap/frame{i}.png','png')

    # ax = sns.heatmap(overlap)
    # plt.show()


In [ ]:
from utils.mydataset import MyDataset
data_dir = 'dataset'
dataset = MyDataset(data_dir=data_dir,num_semi_positives=4)
data_item = dataset[12000]
origin = data_item['origin']
augmen = data_item['augmentation']
semi_posi = data_item['semi_positives']
heatmap = data_item['heatmap']
op_flow = data_item['optical_flow']
print(origin.size())
print(augmen.size())
print(semi_posi.size())
print(heatmap.size())
print(op_flow.size())

In [ ]:
len(semi_posi)

In [ ]:
from utils.mydataset import MyDataset
from torch.utils.data import DataLoader

data_dir = 'dataset'
batch = 3
train_data = MyDataset(data_dir,num_semi_positives=1)
trainloader = DataLoader(train_data, batch_size = batch)

In [ ]:
print(next(iter(trainloader))['origin'])

In [ ]:
import torch
import torch.nn as nn
from model.networks import Encoder, Decoder
from model.losses import contra_loss

encoder = Encoder()
decoder = Decoder()
mse = nn.MSELoss()
optimizer = torch.optim.Adam([{'params':encoder.parameters(),'lr':0.001},
                              {'params':decoder.parameters(),'lr':0.001}])

In [ ]:
keys = ['origin', 'origin_heatmap', 'origin_flow', 'augmentation', 'aug_heatmap', 'aug_flow', 'semi_positives', 'sp_heatmap', 'sp_flow']

In [ ]:
print(len(trainloader))
print(len(trainloader.dataset))

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for train_item in trainloader:
        # get the inputs; data is a list of [inputs, labels]
        for i in range(batch):
            # origin 0.25 처리 필요
            origin = train_item['origin'][i] * 0.25
            h_input = train_item['origin_heatmap'][i]
            f_input = train_item['origin_flow'][i]

            # h_feat = hE(h_input)
            # f_feat = fE(f_input)

            # hf_feat = torch.cat((h_feat, f_feat), dim=1).transpose(1,0)[None,...]
            motion_feature = encoder(h_input,f_input)

            optimizer.zero_grad()

            de_m_feat = decoder(motion_feature)

            # 처음부터 origin 0번 제거하고 가져오기
            loss = mse(origin,de_m_feat)
            loss.backward()
            optimizer.step()
            print(loss)



#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = net(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
#             running_loss = 0.0

# print('Finished Training')

In [ ]:
# if len(origin) == 33:
#     return origin
# elif len(origin) > 33:
#     uniform_num = np.random.uniform(0, len(origin), 33)
#     uniform_origin_keypoint = [origin[int(i)] for i in uniform_num]
#     return np.array(uniform_origin_keypoint)
#     # return np.array(uniform_origin_keypoint)
# elif len(origin) < 33:
#     num_repet = math.ceil((33 - len(origin)) / len(origin) + 1)
#     extended_origin = []
#     for i in range(num_repet):
#         if i % 2 == 0:
#             tmp_origin = origin
#         elif i % 2 == 1:
#             tmp_origin = origin[::-1]
#         extended_origin.append(tmp_origin)
#     extended_origin = np.array(extended_origin).reshape(-1,17,2)[0:33]
#     return extended_origin

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

scaled_origin = []
for i in range(len(tmp_origin)):
    scaled_origin.append(scaler.fit_transform(tmp_origin[i]))
print(np.array(scaled_origin))

In [ ]:
from utils.mydataset import MyDataset
import torch
import numpy as np

data_dir = 'dataset'

train_data = MyDataset(data_dir,num_semi_positives=2)
train_data_0 = train_data[555]
print(train_data_0.keys())

In [ ]:
keys = train_data_0.keys()
for key in keys:
    if key == 'class':
        print('<'+key+'>')
        print(train_data_0[key])
    else:
        print('<'+key+'>')
        print("size: ", train_data_0[key].size())
        print("max: ", train_data_0[key].max())
        print("min: ", train_data_0[key].min())


In [ ]:
from model.networks import Encoder, Decoder
from copy import deepcopy
encoder = Encoder()
encoder_aug = deepcopy(encoder)
encoder_sp = deepcopy(encoder)

In [ ]:
h_input = train_data_0['anchor_heatmap'][None,:]
h_aug_input = train_data_0['aug_heatmap'][None,:]
h_sp_input = train_data_0['sp_heatmap'][None,:]
f_input = train_data_0['anchor_flow'][None,:]
f_aug_input = train_data_0['aug_flow'][None,:]
f_sp_input = train_data_0['sp_flow'][None,:]


In [ ]:
motion_feature = encoder(h_input, f_input)
motion_aug_feature = encoder_aug(h_aug_input, f_aug_input)
motion_sp_feature = encoder_sp(h_sp_input, f_sp_input)
# de_mf = decoder(motion_feature)
# print(de_mf.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# target = torch.Tensor(motion_feature.shape).fill_(73)
# loss = F.cross_entropy(motion_feature, target)

loss = F.cross_entropy(motion_feature, motion_sp_feature)
print(loss)

In [ ]:
print(de_mf.shape)

In [ ]:
print(train_data_0['origin'].max())
print(train_data_0['origin_heatmap'].max())
print(train_data_0['origin_flow'].max())

In [ ]:
print(train_data_0['origin'].min())

In [ ]:
print(train_data_0['origin_heatmap'].size())

In [ ]:
print(train_data_0['origin_flow'].size())

In [ ]:
tmp_origin = train_data_0['origin'] * 0.25
origin_x_max = int(torch.max(tmp_origin[:,:,0]))
origin_y_max = int(torch.max(tmp_origin[:,:,1]))

origin_x_min = int(torch.min(tmp_origin[:,:,0]))
origin_y_min = int(torch.min(tmp_origin[:,:,1]))

print(origin_x_max)
print(origin_y_max)
print(origin_x_min)
print(origin_y_min)

In [ ]:
print(origin_x_max - origin_x_min)
x_center = (origin_x_max + origin_x_min)//2
y_center = (origin_y_max + origin_y_min)//2

print(x_center)
print(y_center)
print(origin_y_max - origin_y_min)
margin = 25

In [ ]:
crop_heatmap = train_data_0['origin_heatmap'][:,:,origin_x_min-margin:origin_x_max+margin,origin_y_min-margin:origin_y_max+margin]

In [ ]:
import torchvision.transforms.functional as ttF
# crop_heatmap.size()
resized_crop_heatmap = ttF.resize(crop_heatmap, (200,100))
print(resized_crop_heatmap.shape)

In [ ]:
import matplotlib.pyplot as plt
for j in range(32):
    tmp_heat = train_data_0['origin_heatmap'][j][0]
    for i in range(1, 17):
        tmp_heat += train_data_0['origin_heatmap'][j][i]
    plt.subplots(1,1)
    plt.imshow(tmp_heat)
    print(tmp_heat.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils.mydataset import MyDataset
from model.networks import Encoder, Decoder
from copy import deepcopy

data_dir = 'dataset'
train_data = MyDataset(data_dir,num_semi_positives=1)
train_data_0 = train_data[555]

origin = train_data_0['origin'].reshape(1,32,17,2) * 0.25
h_input = train_data_0['origin_heatmap']
f_input = train_data_0['origin_flow']

aug_origin = train_data_0['augmentation'].reshape(1,32,17,2) * 0.25
aug_h_input = train_data_0['aug_heatmap']
aug_f_input = train_data_0['aug_flow']

semi_origin = train_data_0['semi_positives'].reshape(-1,32,17,2) * 0.25
semi_h_input = train_data_0['sp_heatmap'].reshape(-1,32,17,270,480)
semi_f_input = train_data_0['sp_flow'].reshape(-1,32,34,270,480)

encoder = Encoder()
decoder = Decoder()

# -- deepcopy init에서 만드는 이유?
cp_encoder1 = deepcopy(encoder)
cp_encoder2 = deepcopy(encoder)

motion_feature = encoder(h_input,f_input)
# (N, 32, 128)
# (N, 128) 전체 비디오 정보를 담음
# de_m_feat = decoder(motion_feature)

aug_feature = cp_encoder1(aug_h_input, aug_f_input)
semi_feature = cp_encoder2(semi_h_input, semi_f_input)

In [ ]:
print(motion_feature.shape)
print(aug_feature.shape)
print(semi_feature.shape)

In [ ]:
mf_mean = motion_feature.mean(dim=2)
af_mean = aug_feature.mean(dim=2)
sf_mean = semi_feature.mean(dim=2)

In [ ]:
sf_mean.shape

In [ ]:
print(origin.shape)
print(de_m_feat.shape)

In [ ]:
print(torch.dot(origin.reshape(-1,2), de_m_feat.reshape(-1,2)))

In [ ]:
reshape_origin = origin.reshape(-1,2)
reshape_feat = de_m_feat.reshape(-1,2)

In [ ]:
similarity = []
for i in range(len(reshape_origin)):
    similarity.append(torch.dot(reshape_origin[i],reshape_feat[i]))
print(torch.tensor(similarity).reshape(-1,1).shape)

In [ ]:
1-(72/73)**9

cross entropy test

In [ ]:
from utils.mydataset import MyDataset
from model.networks import Encoder, Decoder
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
data_dir = 'dataset'
dataset = MyDataset(data_dir=data_dir, num_semi_positives=1)
data_item = dataset[100]
data_item_2 = dataset[200] 
print(data_item.keys())

In [ ]:
encoder = Encoder()
decoder = Decoder()

In [ ]:
origin_anchor = data_item['anchor'][None,:]
h_input = data_item['anchor_heatmap'][None,:]
f_input = data_item['anchor_flow'][None,:]
# class_num = torch.tensor([int(data_item['class'][1:]) - 1 for i in range(73)])[None,:]
# class_num = torch.tensor([int(data_item['class'][1:]) - 1 for i in range(72)])
class_num = torch.tensor([int(data_item['class'][1:]) - 1])
# class_num = []

# class_num = torch.tensor([0 for])

In [ ]:
print(origin_anchor.size())


In [ ]:
class_num.dtype

In [ ]:
print(class_num.size())
print(class_num)

In [ ]:
print(torch.tensor([0,1]).size())

In [ ]:
motion_feature = encoder(h_input, f_input)
decoded = decoder(motion_feature)

In [ ]:
mse = nn.MSELoss()
cE = nn.CrossEntropyLoss()

In [ ]:
test_mf = torch.cat((motion_feature,motion_feature),dim=0)

In [ ]:
print(test_mf.size())

In [ ]:
print(motion_feature.size())

In [ ]:
print(motion_feature.mean(dim=1).shape)
mean_motion_feature = motion_feature.mean(dim=1)

In [ ]:
mse_loss = mse(origin_anchor, decoded)
cE_loss = cE(mean_motion_feature, class_num)

In [ ]:
loss = mse_loss + cE_loss
print(loss)

train

In [ ]:
from utils.mydataset import MyDataset
from model.networks import Encoder, Decoder
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [ ]:
data_dir = 'dataset'
batch = 3
dataset = MyDataset(data_dir, 1)
train_loader = DataLoader(dataset, batch)

In [ ]:
encoder = Encoder()
decoder = Decoder()
mse = nn.MSELoss()
cE = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([{'params':encoder.parameters(),'lr':0.001},
                              {'params':decoder.parameters(),'lr':0.001}])

In [ ]:
for i, train_item in enumerate(train_loader, 0):
    pass

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    
    for i, train_item in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        anchor = train_item['anchor']
        h_input = train_item['anchor_heatmap']
        f_input = train_item['anchor_flow']
        class_num = train_item['class']

        # print(anchor.size())
        # print(h_input.size())
        # print(f_input.size())
        print(class_num, class_num.size())

        motion_feature = encoder(h_input,f_input)
        mean_motion_feature = motion_feature.mean(dim=1)
        decoded = decoder(motion_feature)

        optimizer.zero_grad()

        loss = mse(anchor, decoded) + cE(mean_motion_feature, class_num)
        loss.backward()
        optimizer.step()
        print(loss)

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

In [ ]:
import numpy as np
import pathlib
import os
import json
import math

In [ ]:
def load_data_dir(data_dir):
    all_data_dir_list = []
    action_dir_dic = {}
    label_dic = {}

    for file_path in pathlib.Path(data_dir).glob("*/*.json"):
        action = file_path.name.split('_')[0][file_path.name.split('_')[0].find('A'):]
        all_data_dir_list.append(str(file_path))
        if action in action_dir_dic.keys():
            action_dir_dic[action].append(str(file_path))
        else:
            action_dir_dic[action] = [str(file_path)]
    
    for i, key in enumerate(action_dir_dic.keys()):
        label_dic[key] = i
        
    return all_data_dir_list, action_dir_dic, label_dic

In [ ]:
def extract_keypoints_from_a_json(origin_vid_dir):
        # print("anchor: ", origin_vid_dir)
        with open(origin_vid_dir, "r") as f:
            origin_keypoints = json.load(f)

        origin_keypoints_anno = origin_keypoints['annotations']
        origin_tmp_list = []
        for frame_data in origin_keypoints_anno:
            if frame_data:
                keypoints_list = [frame_data[key] for key in frame_data if frame_data[key]]
                if keypoints_list:
                    origin_tmp_list.extend(keypoints_list)

        # # Nonetype error 발생하는 듯
        # if not origin_tmp_list:
        #     return None

        # reshape x
        origin_anchor_keypoints = np.array(origin_tmp_list)[:,:,[0,1]]
        return origin_anchor_keypoints

In [61]:
def adjusting_frame(origin_keypoint):
    if len(origin_keypoint) == 33:
        return origin_keypoint
    
    elif len(origin_keypoint) > 33:
        # print(len(origin_keypoint))
        linspace_num = np.linspace(start=0, stop=len(origin_keypoint), num=33, endpoint=False)# 적어도 sorting해줄 것 / interpolation or np.linspace로 추출하는 방법 권장
        # print(linspace_num)
        linspace_origin_keypoint = [origin_keypoint[int(i)] for i in linspace_num]
        return np.array(linspace_origin_keypoint)

    elif len(origin_keypoint) < 33:
        num_repet = math.ceil((33 - len(origin_keypoint)) / len(origin_keypoint) + 1)
        extended_origin = []
        for i in range(num_repet):
            if i % 2 == 0:
                tmp_origin = origin_keypoint
            elif i % 2 == 1:
                tmp_origin = origin_keypoint[::-1]
            extended_origin.append(tmp_origin)
        extended_origin = np.array(extended_origin).reshape(-1,17,2)[0:33]
        return extended_origin

In [ ]:
data_dir = 'dataset'

In [64]:
total_dir,dir_by_class,_ = load_data_dir(data_dir)

In [67]:
keys = dir_by_class.keys()
print(len(keys))

73


In [88]:
dir_100 = []
for key in keys:
    tmp_class_value = dir_by_class[key]
    random_num = np.linspace(start=0, stop=len(tmp_class_value), num=100, endpoint=False)
    for i in random_num:
        dir_100.append(tmp_class_value[int(i)])
print(dir_100)

['dataset/a073_kp/S028C002P087R001A073_rgb_kp.json', 'dataset/a073_kp/S026C003P050R002A073_rgb_kp.json', 'dataset/a073_kp/S020C001P041R001A073_rgb_kp.json', 'dataset/a073_kp/S028C002P046R001A073_rgb_kp.json', 'dataset/a073_kp/S027C001P081R001A073_rgb_kp.json', 'dataset/a073_kp/S026C002P049R002A073_rgb_kp.json', 'dataset/a073_kp/S032C001P106R001A073_rgb_kp.json', 'dataset/a073_kp/S029C001P051R002A073_rgb_kp.json', 'dataset/a073_kp/S023C003P055R001A073_rgb_kp.json', 'dataset/a073_kp/S030C002P085R002A073_rgb_kp.json', 'dataset/a073_kp/S030C001P041R002A073_rgb_kp.json', 'dataset/a073_kp/S023C001P063R001A073_rgb_kp.json', 'dataset/a073_kp/S027C003P011R002A073_rgb_kp.json', 'dataset/a073_kp/S020C001P008R002A073_rgb_kp.json', 'dataset/a073_kp/S031C001P042R001A073_rgb_kp.json', 'dataset/a073_kp/S029C003P080R001A073_rgb_kp.json', 'dataset/a073_kp/S029C001P067R002A073_rgb_kp.json', 'dataset/a073_kp/S027C002P082R002A073_rgb_kp.json', 'dataset/a073_kp/S031C001P082R002A073_rgb_kp.json', 'dataset/a0

In [89]:
for i in range(len(dir_100)):
    if 'a001_kp' in dir_100[i]:
        print(dir_100[i])

dataset/a001_kp/S005C001P004R002A001_rgb_kp.json
dataset/a001_kp/S006C003P015R001A001_rgb_kp.json
dataset/a001_kp/S017C002P015R001A001_rgb_kp.json
dataset/a001_kp/S017C002P003R002A001_rgb_kp.json
dataset/a001_kp/S003C001P018R002A001_rgb_kp.json
dataset/a001_kp/S003C002P017R002A001_rgb_kp.json
dataset/a001_kp/S006C002P008R001A001_rgb_kp.json
dataset/a001_kp/S014C001P027R002A001_rgb_kp.json
dataset/a001_kp/S008C001P015R002A001_rgb_kp.json
dataset/a001_kp/S011C003P017R002A001_rgb_kp.json
dataset/a001_kp/S002C003P008R002A001_rgb_kp.json
dataset/a001_kp/S008C003P008R001A001_rgb_kp.json
dataset/a001_kp/S002C003P009R001A001_rgb_kp.json
dataset/a001_kp/S013C002P025R001A001_rgb_kp.json
dataset/a001_kp/S013C001P016R001A001_rgb_kp.json
dataset/a001_kp/S006C001P019R001A001_rgb_kp.json
dataset/a001_kp/S013C003P017R001A001_rgb_kp.json
dataset/a001_kp/S014C002P039R002A001_rgb_kp.json
dataset/a001_kp/S007C003P025R001A001_rgb_kp.json
dataset/a001_kp/S007C002P007R001A001_rgb_kp.json
dataset/a001_kp/S010

In [ ]:
print(total_dir[0])

In [ ]:
data_dir = 'dataset'
total_dir,_,_ = load_data_dir(data_dir)

In [ ]:
import tqdm

In [98]:
for i in tqdm.tqdm(range(len(dir_100))):
    extracted = extract_keypoints_from_a_json(dir_100[i])
    adjusted = adjusting_frame(extracted)
    # heatmap = compute_heatmap(extracted)
    # flow = compute_optical_flow(extracted,heatmap)
    # json_dic = {'file_name':total_dir[i],
    #             'anchor':extracted.tolist(),
    #             'heatmap':heatmap.tolist(),
    #             'flow':flow.tolist()}
    json_dic = {'annotation':adjusted.tolist()}
    save_file_path = dir_100[i].replace('dataset','adjusted_100_dataset')
    if len(adjusted) < 33:
        print(dir_100[i])
    with open(save_file_path, 'w') as f:
        json.dump(json_dic, f)

100%|██████████| 7300/7300 [02:10<00:00, 55.78it/s] 


In [ ]:
adjusted_data_dir = 'adjusted_dataset'

In [ ]:
a_total_dir,_,_ = load_data_dir(adjusted_data_dir)

In [ ]:
def extract_adjusted_kps_from_a_json(origin_vid_dir):
    with open(origin_vid_dir, "r") as f:
        origin_keypoints = json.load(f)

    return np.array(origin_keypoints['annotation'])

In [ ]:
with open(a_total_dir[0], "r") as f:
    origin_keypoints = json.load(f)

origin_keypoints_anno = np.array(origin_keypoints['annotation'])

In [ ]:
print(origin_keypoints_anno.shape)

In [ ]:
extracted = extract_keypoints_from_a_json(total_dir[0])

In [ ]:
print(extracted.shape)

In [ ]:
adjusted = adjusting_frame(extracted)

In [ ]:
json_dic = {'annotation':adjusted.tolist()}

In [ ]:
print(json_dic)

In [ ]:
save_file_path = total_dir[0].replace('dataset','adjusted_dataset')

In [ ]:
print(save_file_path)

In [ ]:
with open(save_file_path, 'w') as f:
    json.dump(json_dic, f)

In [ ]:
print(total_dir[0].split('/S'))

In [97]:
for i in range(len(total_dir)):
    folder_dir = total_dir[i].split('/S')[0].replace('dataset','adjusted_100_dataset')
    try:
        os.makedirs(folder_dir)
    except FileExistsError:
        pass

In [ ]:
def compute_heatmap(keypoints, std=5, rate=0.25):
    points = keypoints.reshape(-1,2) * rate

    x_points = points[:,[0]]
    y_points = points[:,[1]]

    # H = np.tile(np.arange(h), h).reshape(h,h).T
    # W = np.tile(np.arange(reduced_w), reduced_w).reshape(reduced_w,reduced_w)
    # H = np.tile(np.arange(h)[:, None], (h, w))
    # W = np.tile(np.arange(w)[None, :], (h, w))
    H = np.tile(np.arange(270)[:, None], 480)
    W = np.tile(np.arange(480)[:, None], 270).T

    heatmap_list = []
    for i in range(len(x_points)):
        x = x_points[i]
        y = y_points[i]
        # x_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((H-x)/std)**2)
        # y_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((W-y)/std)**2)
        y_gauss = np.exp(-0.5*((H-x)/std)**2)
        x_gauss = np.exp(-0.5*((W-y)/std)**2)
        frame = (x_gauss * y_gauss)
        frame = (frame / frame.max()) * 2 - 1
        heatmap_list.append(frame)
    heatmaps = np.array(heatmap_list).reshape(-1, 17, 270, 480)

    return heatmaps

In [ ]:
heatmap = compute_heatmap(extracted)

In [ ]:
H = np.tile(np.arange(270)[:, None], 480)
print(H)

In [ ]:
print(heatmap[0][0].mean())

In [ ]:
def compute_optical_flow(keypoints, heatmaps, rate=0.25):
        reduced_keypoints = keypoints * rate

        # flow_x = []
        # flow_y = []
        # for i in range(1, len(reduced_keypoints)):
        #     flow_x.append((reduced_keypoints[i] - reduced_keypoints[i-1])[:,[0]])
        #     flow_y.append((reduced_keypoints[i] - reduced_keypoints[i-1])[:,[1]])
        flow = reduced_keypoints[1:] - reduced_keypoints[:-1]
        # flow_x = flow[:,:,[0]]
        # flow_y = flow[:,:,[1]]
        # hw_zeros = torch.zeros(size=(self.h, self.w)) # 빈 판
        # flowmap = torch.zeros((len(flow), 2, self.h, self.w))
        # for i in range(len(flow)):
        #     flowmap[i][0] = flowmap[i][0] + flow[i][0]
        #     flowmap[i][1] = flowmap[i][1] + flow[i][1]

        # heatmaps -> (num_of_frames, num_of_keypoints, H, W)
        # flow -> (num_of_frames - 1, num_of_keypoints, 2)

        # tmp_x = ((hw_zeros + flow_x[i-1][j]) / 270) * 2 - 1
        # tmp_y = ((hw_zeros + flow_y[i-1][j]) / 480) * 2 - 1
        flow = np.stack([
            (flow[..., 0] / 270) * 2 - 1,
            (flow[..., 1] / 480) * 2 - 1
        ], axis=-1)

        # opt_flow_x.append(tmp_x * heatmap[j])
        # opt_flow_y.append(tmp_y * heatmap[j])
        opt_flow = heatmaps[1:, :, :, :, None] * flow[:, :, None, None, :] # (num_of_frames, num_of_keypoints, H, W, 2)
        F, K, H, W, _ = opt_flow.shape
        opt_flow = opt_flow.transpose(0, 1, 4, 2, 3).reshape(-1, K*2, H, W)
        return opt_flow

In [ ]:
heatmap = compute_heatmap(adjusted)
flow = compute_optical_flow(adjusted,heatmap)

In [ ]:
print(heatmap.shape)

In [ ]:
print(flow.shape)

In [ ]:
a = -1
print(max(a,0))

In [ ]:
import numpy as np
import pathlib
import os
import json
import math

In [ ]:
data_dir = 'adjusted_dataset'

In [ ]:
def load_data_dir(data_dir):
    all_data_dir_list = []
    action_dir_dic = {}
    label_dic = {}

    for file_path in pathlib.Path(data_dir).glob("*/*.json"):
        action = file_path.name.split('_')[0][file_path.name.split('_')[0].find('A'):]
        all_data_dir_list.append(str(file_path))
        if action in action_dir_dic.keys():
            action_dir_dic[action].append(str(file_path))
        else:
            action_dir_dic[action] = [str(file_path)]
    
    for i, key in enumerate(action_dir_dic.keys()):
        label_dic[key] = i
        
    # for act_path in os.listdir(data_dir):
    #     if os.path.isdir(os.path.join(data_dir, act_path)):
    #         for file_path in os.listdir(os.path.join(data_dir, act_path)):
    #             if '.json' in file_path:
    #                 whole_file_path = os.path.join(data_dir, act_path, file_path)
    #                 action = file_path.split('_')[0][file_path.split('_')[0].find('A'):]
    #                 all_data_dir_list.append(whole_file_path)
    #                 if action in action_dir_dic.keys():
    #                     action_dir_dic[action].append(whole_file_path)
    #                 else:
    #                     action_dir_dic[action] = [whole_file_path]

    return all_data_dir_list, action_dir_dic, label_dic

In [ ]:
def extract_adjusted_kps_from_a_json(origin_vid_dir):
    with open(origin_vid_dir, "r") as f:
        origin_keypoints = json.load(f)

    return np.array(origin_keypoints['annotation'])

In [ ]:
def compute_heatmap(keypoints, std=5, rate=0.25):
    h = 270
    w = 480
    # points = keypoints.reshape(-1,2) * rate

    # x_points = points[:,[0]]
    # y_points = points[:,[1]]

    # y_points = (keypoints[:, :,[0]] * rate).reshape(-1,1,1)
    # x_points = (keypoints[:, :,[1]] * rate).reshape(-1,1,1)
    y_points = (keypoints[:, :,[0]] * rate).reshape(-1,1) # 561, 1
    x_points = (keypoints[:, :,[1]] * rate).reshape(-1,1) # 561, 1

    # H = np.tile(np.arange(h), h).reshape(h,h).T
    # W = np.tile(np.arange(reduced_w), reduced_w).reshape(reduced_w,reduced_w)
    # H = np.tile(np.arange(h)[:, None], (h, w))
    # W = np.tile(np.arange(w)[None, :], (h, w))
    # H = np.tile(np.arange(h)[:, None], w)
    # W = np.tile(np.arange(w)[:, None], h).T
    
    # H = np.repeat(np.tile(np.arange(h)[:, None], w)[None,:,:], 33*17, axis=0)
    # W = np.repeat(np.tile(np.arange(w)[:, None], h).T[None,:,:], 33*17, axis=0)
    H = np.arange(h)[None, :] # 1, 270
    W = np.arange(w)[None, :] # 1, 480

    # H - y_points: (561, 270)
    log_y_gauss = -0.5 * ((H - y_points)/std)**2 # (561, 270)
    log_x_gauss = -0.5 * ((W - x_points)/std)**2 # (561, 480)
    log_gauss = log_x_gauss[:, None, :] + log_y_gauss[:, :, None]
    gauss = np.exp(log_gauss)
    heatmaps = (gauss / gauss.max(axis=(-1, -2), keepdims=True)) * 2 - 1

    # tmp = np.exp(-0.5*((W-x_points)/std)**2)
    # tmp_y_gauss = np.exp(-0.5*((H-y_points)/std)**2)
    # tmp_x_gauss = np.exp(-0.5*((W-x_points)/std)**2)

    # heatmaps = tmp_x_gauss * tmp_y_gauss
    # print(tmp_y_gauss.shape)
    # print(tmp_y_gauss[0][0])
    # print('x_points: ', x_points[0][0])
    # print(tmp_frame.shape)
    # print(tmp_frame[0])
    # heatmap_list = []
    # for i in range(len(x_points)):
    #     x = x_points[i]
    #     y = y_points[i]
    #     # x_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((H-x)/std)**2)
    #     # y_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((W-y)/std)**2)
    #     y_gauss = np.exp(-0.5*((H-y)/std)**2)
    #     x_gauss = np.exp(-0.5*((W-x)/std)**2)
    #     frame = (x_gauss * y_gauss)
    #     # print(frame.max())
    #     frame = (frame / frame.max()) * 2 - 1
    #     heatmap_list.append(frame)
    heatmaps = heatmaps.reshape(-1, 17, h, w)

    return heatmaps

In [ ]:
# def compute_heatmap(keypoints, std=5, rate=0.25):
#     h = 270
#     w = 480
#     # points = keypoints.reshape(-1,2) * rate

#     # x_points = points[:,[0]]
#     # y_points = points[:,[1]]

#     y_points = (keypoints[:, :,[0]] * rate).reshape(-1,1,1)
#     x_points = (keypoints[:, :,[1]] * rate).reshape(-1,1,1)
#     # y_points = (keypoints[:, :,[0]] * rate).reshape(-1,1) # 561, 1
#     # x_points = (keypoints[:, :,[1]] * rate).reshape(-1,1) # 561, 1

#     # H = np.tile(np.arange(h), h).reshape(h,h).T
#     # W = np.tile(np.arange(reduced_w), reduced_w).reshape(reduced_w,reduced_w)
#     # H = np.tile(np.arange(h)[:, None], (h, w))
#     # W = np.tile(np.arange(w)[None, :], (h, w))
#     # H = np.tile(np.arange(h)[:, None], w)
#     # W = np.tile(np.arange(w)[:, None], h).T
    
#     H = np.repeat(np.tile(np.arange(h)[:, None], w)[None,:,:], 33*17, axis=0)
#     W = np.repeat(np.tile(np.arange(w)[:, None], h).T[None,:,:], 33*17, axis=0)
#     # H = np.arange(h)[None, :] # 1, 270
#     # W = np.arange(w)[None, :] # 1, 480

#     # H - y_points: (561, 270)
#     # log_y_gauss = -0.5 * ((H - y_points)/std)**2 # (561, 270)
#     # log_x_gauss = -0.5 * ((W - x_points)/std)**2 # (561, 480)
#     # log_gauss = log_x_gauss[:, None, :] + log_y_gauss[:, :, None]
#     # gauss = np.exp(log_gauss)
#     # heatmaps = gauss / gauss.sum()

#     tmp = np.exp(-0.5*((W-x_points)/std)**2)
#     tmp_y_gauss = np.exp(-0.5*((H-y_points)/std)**2)
#     tmp_x_gauss = np.exp(-0.5*((W-x_points)/std)**2)

#     heatmaps = tmp_x_gauss * tmp_y_gauss
#     # print(tmp_y_gauss.shape)
#     # print(tmp_y_gauss[0][0])
#     # print('x_points: ', x_points[0][0])
#     # print(tmp_frame.shape)
#     # print(tmp_frame[0])
#     # heatmap_list = []
#     # for i in range(len(x_points)):
#     #     x = x_points[i]
#     #     y = y_points[i]
#     #     # x_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((H-x)/std)**2)
#     #     # y_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((W-y)/std)**2)
#     #     y_gauss = np.exp(-0.5*((H-y)/std)**2)
#     #     x_gauss = np.exp(-0.5*((W-x)/std)**2)
#     #     frame = (x_gauss * y_gauss)
#     #     # print(frame.max())
#     #     frame = (frame / frame.max()) * 2 - 1
#     #     heatmap_list.append(frame)
    # heatmaps = np.array(heatmap_list).reshape(-1, 17, h, w)

#     return heatmaps

In [ ]:
data = load_data_dir(data_dir)

In [ ]:
all_data_dir = data[0]

In [ ]:
one_data_dir = all_data_dir[0]

In [ ]:
print(one_data_dir)

In [ ]:
extracted = extract_adjusted_kps_from_a_json(one_data_dir)
print(extracted.shape)

In [ ]:
heatmap = compute_heatmap(extracted)
# print(heatmap.shape)
# print(heatmap[0].min())

In [ ]:
heatmap = compute_heatmap(extracted)
# print(heatmap.shape)
# print(heatmap[0].min())

In [ ]:
heatmap.shape

In [ ]:
extracted[0][0] * 0.25

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(heatmap[0,0,:,:])

In [ ]:
import numpy as np

def compute_heatmap_test(keypoints, std=5, rate=0.25):
    h = 270
    w = 480
    # points = keypoints.reshape(-1, 2) * rate

    y_points = keypoints[:, :, 0].reshape(-1, 1, 1, 1) * rate
    x_points = keypoints[:, :, 1].reshape(-1, 1, 1, 1) * rate

    H = np.arange(h).reshape(h, 1, 1)
    W = np.arange(w).reshape(1, w, 1)

    y_gauss = np.exp(-0.5 * ((H - y_points) / std) ** 2)
    x_gauss = np.exp(-0.5 * ((W - x_points) / std) ** 2)

    frames = y_gauss * x_gauss
    print(frames.shape)
    print(frames[0])
    # 최대값이 0인 경우를 방지하기 위해 최소값을 설정
    max_values = frames.max(axis=(0, 1), keepdims=True)
    max_values[max_values == 0] = 1

    frames = (frames / max_values) * 2 - 1

    heatmaps = frames.reshape(-1, 17, h, w)

    return heatmaps


In [ ]:
heatmap_test = compute_heatmap_test(extracted)
# print(heatmap_test.shape)
# print(heatmap_test[0].min())

In [ ]:
a = np.array([i+1 for i in range(24)]).reshape(3,2,4)
b = np.array([1,2,3]).reshape(3,1,1)
print(a)
print(b)

In [ ]:
print(a-b)

In [ ]:
H = np.tile(np.arange(270)[:, None], 480)
W = np.tile(np.arange(480)[:, None], 270).T

In [ ]:
# test = np.tile(np.tile(np.arange(270)[:, None], 480)[:,:, None], 33*17)
# test = np.tile(np.arange(270)[None, :, None], (480,33*17))
test = np.repeat(np.tile(np.arange(270)[:, None], 480)[:,:,None],33*17, axis=2)

In [ ]:
print(test.shape)

In [ ]:
print(test[:,:,1])

In [ ]:
# x = (33,17,1).reshape(1,1,-1)

frame matching loss

In [17]:
from tslearn.metrics import dtw_path, dtw

In [22]:
from utils.mydataset import MyDataset
from model.networks import Encoder, Decoder
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from copy import deepcopy
import sys

In [3]:
data_dir = 'adjusted_100_dataset'
batch = 3
dataset = MyDataset(data_dir, 1)
# train_loader = DataLoader(dataset, batch)

In [4]:
encoder = Encoder()
sp_encoder = deepcopy(encoder)
decoder = Decoder()
mse = nn.MSELoss()
cE = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([{'params':encoder.parameters(),'lr':0.001},
                              {'params':decoder.parameters(),'lr':0.001}])

In [5]:
data = dataset[13]
print(data.keys())

(1, 33, 17, 2)
dict_keys(['anchor', 'anchor_heatmap', 'anchor_flow', 'semi_positives', 'sp_heatmap', 'sp_flow', 'class'])


In [6]:
for key in data.keys():
    if key == 'class':
        print(data[key])
    else:
        print(data[key].shape)
        print(data[key].max())
        print(data[key].min())

torch.Size([32, 17, 2])
tensor(0.6255)
tensor(-0.1248)
torch.Size([32, 17, 200, 100])
tensor(0.9999)
tensor(-1.)
torch.Size([32, 34, 200, 100])
tensor(1.0255)
tensor(-1.0186)
torch.Size([1, 32, 17, 2])
tensor(0.7569)
tensor(-0.1932)
torch.Size([1, 32, 17, 200, 100])
tensor(0.9996)
tensor(-1.)
torch.Size([1, 32, 34, 200, 100])
tensor(1.0746)
tensor(-1.0537)
tensor(0)


In [7]:
h_input = data['anchor_heatmap'][None,...]
f_input = data['anchor_flow'][None,...]

In [8]:
h_sp_input = data['sp_heatmap'][None,...]
f_sp_input = data['sp_flow'][None,...]

In [9]:
print(h_sp_input.shape)

torch.Size([1, 1, 32, 17, 200, 100])


In [10]:
motion_feature = encoder(h_input, f_input)
motion_sp_feature = sp_encoder(h_sp_input, f_sp_input)

In [59]:
print(motion_feature.shape)
print(motion_sp_feature.shape)

torch.Size([1, 32, 73])
torch.Size([1, 32, 73])


In [26]:
# dtw_test = dtw_path(motion_feature.detach().squeeze(), motion_sp_feature.detach().squeeze())
dtw_test = dtw_path(motion_feature.squeeze(), motion_sp_feature.squeeze())
# dtw_test = dtw(motion_sp_feature.squeeze())

In [27]:
print(dtw_test)

([(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31)], tensor(2.1729, grad_fn=<SqrtBackward0>))


In [55]:
print(motion_feature[0].size())

torch.Size([32, 73])


In [60]:
print(motion_feature[0, np.array(dtw_test[0])[:,0]].size())
# print(tuple(np.array(dtw_test[0])[:,0]))
print(motion_sp_feature[0, np.array(dtw_test[0])[:,1]].size())

torch.Size([32, 73])
torch.Size([32, 73])


In [30]:
import numpy as np

In [35]:
torch.norm(motion_feature - motion_sp_feature, dim = 2)

tensor([[0.3109, 0.2484, 0.3730, 0.4488, 0.5761, 0.3757, 0.3743, 0.2846, 0.4493,
         0.4475, 0.3065, 0.4472, 0.4736, 0.3155, 0.3501, 0.3237, 0.3501, 0.3480,
         0.2007, 0.3121, 0.3046, 0.4994, 0.4438, 0.3218, 0.2547, 0.3859, 0.5475,
         0.2924, 0.5045, 0.4207, 0.3134, 0.3583]],
       grad_fn=<LinalgVectorNormBackward0>)

In [29]:
from tslearn.metrics import dtw_path_from_metric, SoftDTWLossPyTorch, soft_dtw, dtw_path

In [37]:
import torch
import numpy as np
import sys
e_val = sys.float_info.epsilon
batch = 8
a = torch.randn(8,32,73)
b = torch.randn(8,32,73)

In [66]:
dtw_list = []
for i in range(len(a)):
    tmp_dtw = dtw_path(a[i], b[i])
    tmp_dtw_dist = tmp_dtw[1]
    # print(tmp_dtw_dist)
    tmp_anchor_seq = np.array(tmp_dtw[0])[:,0]
    tmp_sp_seq = np.array(tmp_dtw[0])[:,1]
    # print(tmp_sp_seq)

    anchor_re_tensor = a[i][tmp_anchor_seq]
    # print(anchor_re_tensor.shape)
    sp_re_tensor = b[i][tmp_sp_seq]

    # print(torch.sum(1/(tmp_dtw_dist+e_val))*torch.norm(anchor_re_tensor - sp_re_tensor,dim=1))
    tmp_loss = (1/(tmp_dtw_dist+e_val))*torch.norm(anchor_re_tensor - sp_re_tensor,dim=1)
    # print(tmp_loss)
    tmp_sum_loss = torch.sum(tmp_loss)
    # print(tmp_sum_loss)
    dtw_list.append(tmp_sum_loss)

# print(dtw_list)
dtw_loss = sum(dtw_list)/batch
# print(dtw_loss)

tensor(67.6929)
tensor(67.7875)
tensor(68.8507)
tensor(68.2236)
tensor(67.8157)
tensor(68.2988)
tensor(69.3866)
tensor(68.4552)


In [63]:
soft_dtw_loss = SoftDTWLossPyTorch(gamma=0.1)

In [64]:
soft_dtw_loss(a, b)

tensor([4582.3242, 4595.1440, 4740.4136, 4654.4556, 4598.9692, 4664.7295,
        4814.4941, 4686.1162])

In [65]:
soft_dtw_loss_mean_value = soft_dtw_loss(a, b).mean()
print(soft_dtw_loss_mean_value)

tensor(4667.0811)


In [11]:
from datetime import datetime

In [17]:
now = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')

In [25]:
epoch = 9

In [26]:
f'/data/onwood/sba_project_3D/model_{epoch+1}_{now}.pt'

'/data/onwood/sba_project_3D/model_10_2024-06-19_01:13:01.pt'

In [27]:
from model.losses import frame_matching_loss

In [28]:
fm = frame_matching_loss()

TypeError: frame_matching_loss() missing 2 required positional arguments: 'anchor' and 'sp'